<a href="https://colab.research.google.com/github/wlgnsthd/BigdataAnalysis/blob/main/%EA%B3%B5%EA%B3%B5%EB%8D%B0%EC%9D%B4%ED%84%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#import

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import math
import pandas as pd
from dateutil.relativedelta import relativedelta

result_pd = pd.DataFrame()

traffic_num = 1 #traffic 적절하게 수정

access_key = "39Eri2U1%2BFHElpAAe2Cqjlkbm9Cup%2BTfY%2BQ4I5aAR6Ho5yEyM9Dbng00rv%2B%2FfBVh3m4bsfMnxRYP4yjPBmTy7Q%3D%3D" 

#데이터 전처리

In [ ]:
start_time = []
start = datetime.datetime.strptime(str(20200101), "%Y%m%d") # 수집 시작 날짜 (수정)
end = datetime.datetime.strptime(str(20201231), "%Y%m%d") # 수집 종료 날짜 + 1 (수정)
date_generated = [start + relativedelta(months=1)*x for x in range(0,12)] #monthly

for date in date_generated:
    start_time.append(date.strftime("%Y%m%d"))

In [ ]:
def get_request_url(url):
    req = urllib.request.Request(url)    
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
            
    except Exception as e:
        return None

#데이터 처리

##일반민원

In [ ]:
def getname(month):
    end_point = "http://apis.data.go.kr/1140100/minAnalsInfoView3/minTopNKeyword3"
    parameters = "?serviceKey=" + access_key
    parameters += "&resultCount=" + str(traffic_num)
    parameters += "&dateFrom=" + str(month)
    parameters += "&dateTo=" + (datetime.datetime.strptime(month, "%Y%m%d") + relativedelta(months=1)-datetime.timedelta(days=1)).strftime("%Y%m%d")##끝나는날짜
    parameters += "&target=pttn" 
    parameters += "&mainSubCode=1620000" # 국가인권위원회
    url = end_point + parameters
    print(url)
    retData = get_request_url(url)

    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [ ]:
transfer_label = []
transfer_value = []
for i in start_time:
    transfer_l=[]
    transfer_v=[]
    for n in range(traffic_num):
        try:
            b = getname(i)
            print(b)
            transfer_l.append(b[n]['label'])
            transfer_v.append(b[n]['value'])
        except:
            continue
    transfer_label.append(transfer_l)   
    transfer_value.append(transfer_v)
#print(transfer_label)
#print(transfer_value)    

In [ ]:
for i in range(len(start_time)):
    if not transfer_label[i]:
        transfer_label[i].append("None")
    if not transfer_value[i]:
        transfer_value[i].append("None")

In [ ]:
for k in range(len(start_time)):
    for v in range(len(transfer_label[k])):
        weeknum= k+1
        append_pd = pd.DataFrame({"월별":weeknum,"민원별":"일반민원","키워드":transfer_label[k][v],"빈도":transfer_value[k][v]},index=[k])
        result_pd = result_pd.append(append_pd)

In [ ]:
display(pd.DataFrame(result_pd))

##고충민원

In [ ]:
def getname_2(month):
    end_point = "http://apis.data.go.kr/1140100/minAnalsInfoView3/minTopNKeyword3"
    parameters = "?serviceKey=" + access_key
    parameters += "&resultCount=" + str(traffic_num)
    parameters += "&dateFrom=" + str(month)
    parameters += "&dateTo=" + (datetime.datetime.strptime(month, "%Y%m%d") + relativedelta(months=1)-datetime.timedelta(days=1)).strftime("%Y%m%d")##끝나는날짜
    parameters += "&target=dfpt" 
    parameters += "&mainSubCode=1620000" # 국가인권위원회
    url = end_point + parameters
    print(url)
    retData = get_request_url(url)

    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [ ]:
transfer_label = []
transfer_value = []
for i in start_time:
    transfer_l=[]
    transfer_v=[]
    for n in range(traffic_num):
        try:
            b = getname_2(i)
            print(b)
            transfer_l.append(b[n]['label'])
            transfer_v.append(b[n]['value'])
        except:
            continue
    transfer_label.append(transfer_l)   
    transfer_value.append(transfer_v)
print(transfer_label)
print(transfer_value)

In [ ]:
for i in range(len(start_time)):
    if not transfer_label[i]:
        transfer_label[i].append("None")
    if not transfer_value[i]:
        transfer_value[i].append("None")

In [ ]:
for k in range(len(start_time)):
    for v in range(len(transfer_label[k])):
        weeknum= k+1
        append_pd = pd.DataFrame({"월별":weeknum,"민원별":"고충민원","키워드":transfer_label[k][v],"빈도":transfer_value[k][v]},index=[k])
        result_pd = result_pd.append(append_pd)

In [ ]:
display(pd.DataFrame(result_pd))

#출력

In [ ]:
result_pd=result_pd.sort_values(['월별'],ascending=True)
display(pd.DataFrame(result_pd))
result_pd.to_csv("./result_openapi_20171648_JihoonSong.csv",sep=",",na_rep='NaN',index=False, encoding = 'utf-8-sig')